In [1]:
!pwd
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount = True)

/content
Mounted at /content/drive


In [2]:
pip install yfinance pandas

In [6]:
# ==============================================================================
# CELL 3: Main script to create Google Sheets with GOOGLEFINANCE formula
# ==============================================================================

import pandas as pd
from datetime import datetime

# --- Configuration ---
# Set the path to your input CSV file in Google Drive.
input_csv_path = '/content/drive/MyDrive/_invest2024/DJIndexes_data/djindexes_googlesheets.csv'

# Set the desired output folder paths in your Google Drive.
output_folders = {
    'daily': '_invest2024/DJIndexes_data/data/daily',
    'weekly': '_invest2024/DJIndexes_data/data/weekly',
    'monthly': '_invest2024/DJIndexes_data/data/monthly'
}

# IMPORTANT: Set this to your email address so the new sheets are shared with you.
USER_EMAIL = 'imagda@gmail.com'   # <-- CHANGE THIS TO YOUR EMAIL

def get_or_create_folder(folder_path):
    """Finds a folder by path, creating it if it doesn't exist. Returns folder ID."""
    parent_id = 'root'
    folders = folder_path.split('/')
    for folder_name in folders:
        query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and '{parent_id}' in parents and trashed=false"
        response = drive_service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
        results = response.get('files', [])
        if results:
            parent_id = results[0].get('id')
        else:
            print(f"Creating folder: {folder_name}")
            folder_metadata = {
                'name': folder_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_id]
            }
            folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
            parent_id = folder.get('id')
    return parent_id



# --- Main Logic ---

# 1. Read the list of tickers from your CSV file.
try:
    print(f"\nReading tickers from: {input_csv_path}")
    df_tickers = pd.read_csv(input_csv_path, header=None)
    tickers = df_tickers[1].tolist() # Access the first column explicitly
    print(f"Successfully found {len(tickers)} tickers.")
except FileNotFoundError:
    print(f"ERROR: File not found at '{input_csv_path}'.")
    print("Please make sure you have placed the file in the correct Google Drive path.")
else:
    # 2. Define the date range
    end_date = datetime.now()
    start_date = datetime(2024, 1, 1)

    # 3. Process each interval (daily, weekly, monthly)
    intervals = ['DAILY', 'WEEKLY']  # Note: Google Finance doesn't support MONTHLY directly

    for interval in intervals:
        interval_name = interval.lower()
        print(f"\n=== Processing {interval_name.upper()} data ===")

        # Get the destination folder ID for this interval
        print(f"Ensuring output folder exists: '{output_folders[interval_name]}'")
        destination_folder_id = get_or_create_folder(output_folders[interval_name])
        print(f"All {interval_name} sheets will be saved in folder ID: {destination_folder_id}")

        # 4. Loop through each ticker, create a new Google Sheet
        print(f"\nStarting {interval_name} data sheet creation process...")
        for ticker in tickers:
            # Create the sanitized ticker name for the GOOGLEFINANCE formula (without '^')
            sanitized_ticker = ticker.replace('^', '')

            try:
                print(f"  -> Processing {interval_name} data for ticker: {ticker}...")

                # Create the Google Sheet in the desired Drive folder
                sheet_name = f"{ticker}_{interval_name}"
                spreadsheet = gc.create(sheet_name, folder_id=destination_folder_id)
                worksheet = spreadsheet.sheet1

                # Use separate formulas with proper column spacing to avoid array conflicts
                # Place each formula with enough space between columns (every 3rd column)
                formulas_data = [
                    ('A1', f'=GOOGLEFINANCE("{sanitized_ticker}", "open", DATE({start_date.year},{start_date.month},{start_date.day}), TODAY(), "{interval}")'),
                    ('D1', f'=GOOGLEFINANCE("{sanitized_ticker}", "high", DATE({start_date.year},{start_date.month},{start_date.day}), TODAY(), "{interval}")'),
                    ('G1', f'=GOOGLEFINANCE("{sanitized_ticker}", "low", DATE({start_date.year},{start_date.month},{start_date.day}), TODAY(), "{interval}")'),
                    ('J1', f'=GOOGLEFINANCE("{sanitized_ticker}", "close", DATE({start_date.year},{start_date.month},{start_date.day}), TODAY(), "{interval}")'),
                    ('M1', f'=GOOGLEFINANCE("{sanitized_ticker}", "volume", DATE({start_date.year},{start_date.month},{start_date.day}), TODAY(), "{interval}")')
                ]

                # Prepare batch updates with proper spacing
                updates = []
                for cell_range, formula in formulas_data:
                    updates.append({
                        'range': cell_range,
                        'values': [[formula]],
                        'majorDimension': 'ROWS'
                    })

                # Use batch_update to set all formulas at once
                worksheet.batch_update(updates, value_input_option='USER_ENTERED')

                # Share the sheet so you have access
                spreadsheet.share(USER_EMAIL, perm_type='user', role='writer')

                print(f"     - Success! {interval_name.capitalize()} sheet created. Link: {spreadsheet.url}")

            except Exception as e:
                print(f"     - ERROR: Failed to process {ticker} for {interval_name} data. Reason: {e}")

    print("\nScript finished. All formulas have been inserted into your specified Google Drive folders.")


Reading tickers from: /content/drive/MyDrive/_invest2024/DJIndexes_data/djindexes_googlesheets.csv
Successfully found 106 tickers.

=== Processing DAILY data ===
Ensuring output folder exists: '_invest2024/DJIndexes_data/data/daily'
Creating folder: data
Creating folder: daily
All daily sheets will be saved in folder ID: 12QlN6UMqVgGsiAHtN3axu-MUVQfM9ImH

Starting daily data sheet creation process...
  -> Processing daily data for ticker: ticker...
     - Success! Daily sheet created. Link: https://docs.google.com/spreadsheets/d/15nASyt2gY9mqqnI6qu5dq79UDNU3tlLecPegiDeWUQw
  -> Processing daily data for ticker: ^DJUSBM...
     - Success! Daily sheet created. Link: https://docs.google.com/spreadsheets/d/1iCzW9pfisudoLwold4V0JjUE9ayeKRgNS_yuQCPUXVY
  -> Processing daily data for ticker: ^DJUSBT...
     - Success! Daily sheet created. Link: https://docs.google.com/spreadsheets/d/1_481Z4OmHFnWB9QJXHiGZm1tqXx8QJAZujUTCn0AdQc
  -> Processing daily data for ticker: ^DJUSCY...
     - Success!